In [41]:
import numpy as np
import random

from keras.models import Sequential
from keras.layers import Dropout, Dense
from keras.optimizers import SGD

import nltk
from nltk.stem import WordNetLemmatizer
#nltk.download("punkt")
#nltk.download('wordnet')
#nltk.download('omw-1.4')
import json
import string

In [43]:
#The "intents" file could also be written directly in the notebook.
with open(file = "Intents.json") as data:
    data = data.read()

intents = json.loads(s = data)

In [44]:
#A list in which all the tokens of each pattern will be stored. Will be referred to as the vocabulary.
words = []

#A list in which all the tags will be stored.
classes = []

#A list in which all the patterns and their related tag will be stored as pairs.
documents = []

#The tokenization process will be performed on each pattern.
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        
        #Will perform the tokenization on each pattern.
        tokens = nltk.word_tokenize(text = pattern)
        words.extend(tokens)
        documents.append((tokens, intent["tag"]))
        
    #Will append each tag to the “classes” list if it is not there already.
    if intent["tag"] not in classes: classes.append(intent["tag"])

In [45]:
lemmatizer = WordNetLemmatizer()

#List comprehension is being used to update the "words" list with the lemmatized lowercased version of each word that is not
#a punctuation.
words = [lemmatizer.lemmatize(word = word.lower()) for word in words if word not in string.punctuation]
words = sorted(set(words))
classes = sorted(set(classes))

#Both lists could also be saved as txt instead of json.
json.dump(obj = words, fp = open(file = "Words.json", mode = "w"))
json.dump(obj = classes, fp = open(file = "Classes.json", mode = "w"))

In [46]:
training = []
output = np.zeros(shape = len(classes), dtype = "int")

#Creation of the Bag of Words model.
for document in documents:
    #A list for the encoded version of each pattern.
    bag_of_words = []
    
    #Will extract each pattern's list of tokens.
    pattern_words = document[0]
    
    #List comprehension is being used to update the "pattern_words" list with the lemmatized lowercased version of each word.
    #We have to lemmatize the pattern words, because the words in the vocabulary (The “words” list) are also lemmatized, 
    #and therefore we can compare oranges to oranges.
    pattern_words = [lemmatizer.lemmatize(word = word.lower()) for word in pattern_words]
    
    #The encoding will be performed upon each pattern.
    for word in words:
        bag_of_words.append(1) if word in pattern_words else bag_of_words.append(0)
        
    #For every current considered pattern, the index of the tag the pattern belongs to will be marked with 1. 
    #This is for the neural network to learn later what kind of patterns belong to what kind of tag.
    y = list(output)
    y[classes.index(document[1])] = 1
    
    #Every encoded pattern-tag pair will be stored as training data for the neural network.
    training.append([bag_of_words, y])
    
#Will shuffle the training data to avoid optional biases.
random.shuffle(x = training)

#Will convert the “training” list into a numpy array, so we can split the data to x and y later on.
training = np.array(object = training, dtype = "object")

#We have to perform the conversion into numpy array, so we can feed it into the neural network later on.
X_train = np.array(object = list(training[:, 0]))
y_train = np.array(object = list(training[:, 1]))

In [61]:
ann = Sequential()

#The "input_shape" parameter accepts a tuple form values, so there is a need to "convert" the length size into a tuple.
ann.add(layer = Dense(units = 128, activation = "relu", input_shape = (len(X_train[0]),)))
ann.add(layer = Dropout(rate = 0.5))

ann.add(layer = Dense(units = 64, activation = "relu"))
ann.add(layer = Dropout(rate = 0.3))

ann.add(layer = Dense(units = len(y_train[0]), activation = "softmax"))

In [62]:
optimizer = SGD(learning_rate = 0.01, momentum = 0.9, nesterov = True)

The cross entropy cost (or loss) function has been chosen, because we are predicting the likelihood of an example belonging to one of more than two classes, and the choice of the cost (or loss) function is directly related to the activation function we chose in the output layer of our neural network, which is the softmax activation function

The labels expected to be provided to the neural network as OneHot and not integers, and hence the categorical_crossentropy has been selected.

In [63]:
ann.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics = ["accuracy"])

In [64]:
training = ann.fit(x = X_train, y = y_train, batch_size = 30, epochs = 200, verbose = 1)

Epoch 1/200
1/1 [==============================] - 0s 358ms/step - loss: 1.8099 - accuracy: 0.1667
Epoch 2/200
1/1 [==============================] - 0s 4ms/step - loss: 1.7887 - accuracy: 0.3000
Epoch 3/200
1/1 [==============================] - 0s 3ms/step - loss: 1.7685 - accuracy: 0.2333
Epoch 4/200
1/1 [==============================] - 0s 3ms/step - loss: 1.7449 - accuracy: 0.2667
Epoch 5/200
1/1 [==============================] - 0s 4ms/step - loss: 1.8276 - accuracy: 0.1667
Epoch 6/200
1/1 [==============================] - 0s 3ms/step - loss: 1.7869 - accuracy: 0.1667
Epoch 7/200
1/1 [==============================] - 0s 5ms/step - loss: 1.7773 - accuracy: 0.1667
Epoch 8/200
1/1 [==============================] - 0s 3ms/step - loss: 1.7666 - accuracy: 0.2667
Epoch 9/200
1/1 [==============================] - 0s 4ms/step - loss: 1.7233 - accuracy: 0.2000
Epoch 10/200
1/1 [==============================] - 0s 3ms/step - loss: 1.7519 - accuracy: 0.1667
Epoch 11/200
1/1 [=========

1/1 [==============================] - 0s 3ms/step - loss: 0.1291 - accuracy: 0.9667
Epoch 168/200
1/1 [==============================] - 0s 3ms/step - loss: 0.1158 - accuracy: 1.0000
Epoch 169/200
1/1 [==============================] - 0s 3ms/step - loss: 0.2113 - accuracy: 0.9667
Epoch 170/200
1/1 [==============================] - 0s 2ms/step - loss: 0.1520 - accuracy: 0.9667
Epoch 171/200
1/1 [==============================] - 0s 2ms/step - loss: 0.1047 - accuracy: 1.0000
Epoch 172/200
1/1 [==============================] - 0s 3ms/step - loss: 0.1148 - accuracy: 1.0000
Epoch 173/200
1/1 [==============================] - 0s 3ms/step - loss: 0.1294 - accuracy: 1.0000
Epoch 174/200
1/1 [==============================] - 0s 2ms/step - loss: 0.1385 - accuracy: 0.9667
Epoch 175/200
1/1 [==============================] - 0s 3ms/step - loss: 0.1420 - accuracy: 1.0000
Epoch 176/200
1/1 [==============================] - 0s 3ms/step - loss: 0.1711 - accuracy: 0.9333
Epoch 177/200
1/1 [=====

In [65]:
ann.save(filepath = "final_model.h5")